## 🎯 Project Goal

The goal of this project is to **detect whether a Persian comment mentions price or not**.  
Using text preprocessing (normalization, tokenization, stopword removal) and a **Naive Bayes classifier** based on word counts, the model predicts:

- **0:** No mention of price  
- **1:** Mentions price  

This project demonstrates a simple **NLP text classification** pipeline for Persian language data.


## 1. Import Libraries

Import essential libraries for data handling, text preprocessing, and model evaluation:  
- **NumPy, Pandas** for data manipulation  
- **accuracy_score** from `sklearn` for evaluating predictions  
- **Hazm** tools for Persian text normalization and tokenization


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from hazm import Normalizer, word_tokenize, stopwords_list

## 2. Load Dataset

Read the training and test datasets from CSV files using Pandas.  
Display the training data to inspect its structure and available columns.


In [4]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
train_data

,comment,price_value
0,قیمت مناسب وکیفیت خوب پیشنهادمیکنم حتما خرید کنید,1
1,به اندازه یک میلیمتر دورتادور گوشی خالی میماند...,0
2,از همه نظر عالی و یک خرید خوب در قیمت حدود۴۰ ...,1
3,فقط یک بار هر یک ربع ساعت 1 درصد شارژ کرد بعدش...,0
4,قیمت این کالا خیلی تغییر میکنه . من خریدم چندر...,1
...,...,...
39995,خیلی خوبه واسه گوشی m20ولی یه ترک از پایین داش...,0
39996,چند روزه دارم استفاده میکنم در یک کلام عالیه\r...,1
39997,من سی تومن خریدم و با توجه به قیمت ارزش خرید د...,1
39998,عالیه از هر نظر \nهم قیمتش کمه \nهم قطعاتش زیا...,0


## 3. Text Preprocessing

Normalize and tokenize Persian comments using Hazm.  
Remove common stopwords to keep only meaningful words for analysis.  
Apply preprocessing to both train and test comment columns.


In [5]:
norm = Normalizer()
stops = set(stopwords_list())
def preprocessing(text):
    filtered = []
    txt_norm = norm.normalize(text)
    token = word_tokenize(txt_norm)
    filtered = [word for word in token if word not in stops]
    return filtered

train_data['comment'] = train_data['comment'].apply(preprocessing)
test_data['comment'] = test_data['comment'].apply(preprocessing)

In [ ]:
train_data.head()

In [7]:
test_data.head()

,comment
0,"[قیمت, جنس, نمیشه, ،, واقعا, راضی‌ام, ازش]"
1,"[شیکه, جنسش, خوبه, قابل‌استفاده, توصیه]"
2,"[میشه, G, ۲, Ts, بهتره]"
3,"[شکل, داره, راضیم, واسه, جهیزیه, دخترم, خریدم]"
4,"[توجه, گوشی, مادرم, خریدم, توقع, عملکرد, ویژه‌..."


## 4. Prior Probability Calculation  

Compute the prior probabilities for each class (`price_value = 0` or `1`).  
This represents how frequently each label appears in the training data —  
essentially, the baseline likelihood of each class before observing any text features.  

- `p0`: Probability that the comment **does not** mention price.  
- `p1`: Probability that the comment **does** mention price.  

These values are stored in a dictionary `prior_probability` for later use in the Naive Bayes model.


In [9]:
p0 = train_data['price_value'].apply(lambda x : x == 0).sum() / train_data['price_value'].count()
p1 = train_data['price_value'].apply(lambda x : x == 1).sum() / train_data['price_value'].count()

prior_probability = {
    0 : p0,
    1 : p1
}

## 5. Token Counting Function  

Define a helper function `token_counter` that counts how many times each token (word) appears in a given list of tokens.  

- **Input:** A list of tokenized words (e.g., a comment after preprocessing).  
- **Process:**  
  - Iterate over each token.  
  - If the token already exists in `count_dict`, increment its count.  
  - Otherwise, initialize it with a count of 1.  
- **Output:** A dictionary where keys are tokens and values are their frequencies.  

This function will later help compute word occurrence statistics for each class (price-related or not).


In [10]:
def token_counter(texts):
    count_dict = {}
    for token in texts:
        if token in count_dict:
            count_dict[token] += 1
        else:
            count_dict[token] = 1
    return count_dict

## 6. Counting Words in Class 0 (No Price Mention)

Extract all comments with `price_value = 0` (no price mentioned) and count word frequencies for this class.  
- For each comment, count words using `token_counter`.  
- Aggregate counts into `negative_class_count` for later probability calculations.


In [11]:
comments = train_data[train_data['price_value'] == 0]['comment']
negative_class_count = {}
for text in comments:
    word_counts = token_counter(text)
    for word, count in word_counts.items():
        if word in negative_class_count:
            negative_class_count[word] += count
        else:
            negative_class_count[word] = count
negative_class_count

{'اندازه': 667,
 'میلیمتر': 18,
 'دورتادور': 3,
 'گوشی': 944,
 'خالی': 113,
 'می\u200cماند': 6,
 'قاب': 173,
 'استفاده': 2797,
 'پوشش': 72,
 'یک\u200cبار': 68,
 'ربع': 6,
 'ساعت': 333,
 '۱': 198,
 'درصد': 93,
 'شارژ': 451,
 'بعدش': 64,
 'دیگه': 1011,
 'کلا': 380,
 'نکرد': 106,
 'راحت': 508,
 'نرم': 394,
 'زیبا': 467,
 'قیمت': 497,
 'نکردم': 419,
 'ریز': 118,
 'هستش': 563,
 'هفته': 199,
 'هست': 2956,
 'خریدم': 1745,
 'تبلتهای': 1,
 'موجود': 155,
 'بازار': 172,
 'خریدش': 181,
 'مقرون': 6,
 'بصرفه': 11,
 '،': 5719,
 'صفحه': 254,
 'نمایش': 73,
 'کیفیت': 2340,
 'سرعت': 194,
 'هنگی': 7,
 'کارای': 42,
 'کامپیوترو': 1,
 'میشه': 2072,
 'انجام': 139,
 'موبایل': 92,
 'مطالعه': 50,
 '.': 12160,
 'مناطق': 5,
 'سردسیر': 1,
 'هوای': 17,
 'سرد': 36,
 'رادیاتور': 3,
 'نمیخوره': 213,
 'نتیجه': 54,
 'بخاری': 13,
 'ماشین': 201,
 'خوبه': 3290,
 'گرم': 181,
 'میکنه': 1141,
 'اگه': 721,
 'معتدل': 3,
 'هستین': 22,
 'زمستان': 22,
 'وقته': 28,
 'گرفتم': 1127,
 'ازش': 759,
 'راضی': 1216,
 'هستم': 523,
 'ترس': 11

## 7. Counting Words in Class 1 (Price Mentioned)

Collect comments where `price_value = 1` (price mentioned) and count word occurrences:  
- Use `token_counter` for each comment.  
- Combine results into `positive_class_count` — a dictionary of word frequencies for the “price mentioned” class.


In [12]:
comments = train_data[train_data['price_value'] == 1]['comment']
positive_class_count = {}
for text in comments:
    word_counts = token_counter(text)
    for word, count in word_counts.items():
        if word in positive_class_count:
            positive_class_count[word] += count
        else:
            positive_class_count[word] = count
        
positive_class_count

{'قیمت': 7737,
 'وکیفیت': 30,
 'پیشنهادمیکنم': 10,
 'حتما': 886,
 'خرید': 2484,
 '۴۰': 108,
 'تومن': 1069,
 'کالا': 1782,
 'میکنه': 774,
 '.': 12214,
 'خریدم': 2914,
 'چندروز': 4,
 'قیمتش': 5156,
 'رنگش': 319,
 'خوشرنگ': 46,
 'هست': 2846,
 'بخصوص': 28,
 'شماره\u200cاش': 1,
 'پوستهایی': 1,
 'گندمی': 1,
 'می\u200cخوان': 6,
 'روشن\u200cتر': 4,
 'بشه': 363,
 'عکس': 498,
 'می\u200cبینید': 38,
 'سوکت\u200cها': 3,
 'جنس': 1405,
 'پلاستیک': 194,
 'رنگ\u200cآمیزی': 5,
 'ضعیف': 141,
 'یه': 1809,
 'سوکتشم': 1,
 'ک': 584,
 'اصلا': 1371,
 'کج': 36,
 'بدنه': 90,
 'سیم': 93,
 'نوشته': 83,
 'ساخت': 193,
 'چین': 29,
 'مدل': 446,
 'دیگه': 916,
 'کابل': 110,
 '۶۰': 78,
 'اینا': 84,
 '،': 5753,
 '۱۰۰': 126,
 'زده_بود': 34,
 'گفتم': 88,
 'اصلی': 164,
 'باشه': 1070,
 'اینطور': 27,
 'استفاده': 1591,
 'کیفیت': 3212,
 'داره': 4850,
 'تاچ': 20,
 'میشه': 1670,
 'مجبور': 75,
 'میشی': 11,
 'بکنی': 3,
 'پرایویسی': 1,
 'واقعا': 2061,
 'کار': 1087,
 'حیف': 145,
 'چسبش': 33,
 'خراب': 194,
 'پولمو': 23,
 'ریختم': 34,
 

## 8. Computing Word Probabilities for Each Class (Naive Bayes)

This section defines how to calculate the likelihood of a comment belonging to each class using the **Multinomial Naive Bayes** formula.

- `total_pos` / `total_neg`: total word counts for each class.  
- `vocab` and `V`: combined vocabulary and its size (for smoothing).  
- `compute_probability(text, cls)`:  
  - Starts with the **prior probability** for class `cls`.  
  - Applies **Laplace smoothing** (`+1` in numerator, `+V` in denominator).  
  - Multiplies probabilities of each word raised to its frequency power (`prob ** count`).  
  - Returns the final likelihood for the comment belonging to class `cls`.


In [13]:
total_pos = sum(positive_class_count.values())
total_neg = sum(negative_class_count.values())
vocab = set(list(positive_class_count.keys()) + list(negative_class_count.keys()))
V = len(vocab)

def compute_probability(text, cls):
    mul = prior_probability[cls]  
    if cls == 0:
        total_count = total_neg + V 
        class_counts = negative_class_count
    else:
        total_count = total_pos + V
        class_counts = positive_class_count
        
    for word, count in text.items():
        word_count_in_class = class_counts.get(word, 0) + 1
        prob = word_count_in_class / total_count
        mul *= prob ** count
        
    return mul

## 9. Predicting Comment Class

The `predict()` function applies the **Naive Bayes rule** to classify each comment:

- Converts each comment to a bag-of-words (`token_counter`).
- Calculates probabilities for both classes (`0` and `1`).
- Assigns the class with the higher probability.

➡️ Returns all predictions as a NumPy array.


In [14]:
def predict(test):
    predictions = []
    for txt in test:
        bow = token_counter(txt) 
        com_0 = compute_probability(bow, 0)
        com_1 = compute_probability(bow, 1)
        if com_0 > com_1:
            predictions.append(0)
        else:
            predictions.append(1) 
    return np.array(predictions)

## 10. Evaluating and Generating Predictions


In [15]:
train_predictions = predict(train_data['comment'])
accuracy_score(train_data['price_value'], train_predictions)

0.896075

In [17]:
submission = pd.DataFrame()
submission['comment'] = test_data['comment']
submission['price_value'] = predict(test_data['comment'])
submission

,comment,price_value
0,"[قیمت, جنس, نمیشه, ،, واقعا, راضی‌ام, ازش]",1
1,"[شیکه, جنسش, خوبه, قابل‌استفاده, توصیه]",1
2,"[میشه, G, ۲, Ts, بهتره]",1
3,"[شکل, داره, راضیم, واسه, جهیزیه, دخترم, خریدم]",0
4,"[توجه, گوشی, مادرم, خریدم, توقع, عملکرد, ویژه‌...",1
...,...,...
7995,"[کرم, مرطوب‌کننده‌ای, هست, برا, دست, الان, خریدم]",0
7996,"[شگفت‌انگیز, ارزش, خرید, داره, خداییش, عطر, خو...",1
7997,"[شیشه, درآوردم, آبش, خالی, گذاشتم, ترازو, ،, ۴...",1
7998,"[اصلا, اونی, ک, سفارش, دادم, یه, مارک, دیگه, ی...",0
